In [10]:
!pip install requests

In [11]:
import base64
import requests
import datetime
from urllib.parse import urlencode
from urllib.parse import urlparse
import webbrowser

In [12]:
client_id = 'f7d3070bc538436e9d424e6954776192'
client_secret = '44d4c5c3a6e4458b999d02206383a16e'

In [23]:
class SpotifyAPI():
    access_token = None
    access_token_expires = None
    access_token_did_expire = True
    client_id = None
    client_secret = None
    did_perform_grant_flow_auth = False
    authorize_url = "https://accounts.spotify.com/authorize"
    redirected_url = None
    
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("client_secret or client_id is None")
        client_credentianls = f"{client_id}:{client_secret}"
        client_credentials_b64 = base64.b64encode(client_credentials.encode())
        return client_credentials_b64.decode()
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def get_token_headers(self):
        return {
            "Authorization": f"Basic {client_credentials_b64.decode()}"
        }
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        request = requests.post(token_url, data=token_data, headers=token_headers)
        if request.status_code in range(200,299):
            token_response_data = request.json()
            current_time = datetime.datetime.now()
            access_token = token_response_data['access_token']
            expires_in_seconds = token_response_data['expires_in']
            expires = current_time + datetime.timedelta(seconds=expires_in_seconds)
            self.access_token = access_token
            self.access_token_expires = expires
            self.access_token_did_expire = expires < current_time
            return True
        return False
    
    def perform_auth_grant_flow(self):
        client_id = self.client_id
        authorize_url = self.authorize_url
        redirect_uri = "http://localhost:8888"
        authorize_url = self.authorize_url
        scope = "user-read-playback-position user-read-recently-played"
        parameters = urlencode({"client_id": client_id, "redirect_uri": redirect_uri, "response_type": "token", 
                                "scope": scope})
        request = requests.get(f"{authorize_url}?{parameters}")
        if request.status_code in range(200,299):
            webbrowser.open(f"{authorize_url}?{parameters}")
            return True
        else:
            raise Exception("failed to request")
            return False
    
    def set_redirected_url(self, url):
        self.redirected_url = url
        
    def get_redirected_url(self):
        return self.redirected_url
    
    def parse_redirected_url(self, url):
        if did_perform:
            spotify.set_redirected_url(url)
            query = urlparse(spotify.get_redirected_url())[5]
            parts = query.split('&')
    
        if range(len(parts) != 3):
            raise Exception("Access denied")
    
        for i in range(len(parts)):
            parts[i] = parts[i].split('=')[1]
        return parts
    
    def set_access_token_and_expire(self, parts):
        self.access_token = parts[0]
        expires_in_seconds = int(parts[2])
        current_time = datetime.datetime.now()
        expires = current_time + datetime.timedelta(seconds=expires_in_seconds)
        self.token_expires = expires
        self.access_token_did_expire = expires < current_time
    
    def get_access_token(self):
        return self.access_token

In [24]:
spotify = SpotifyAPI(client_id, client_secret);
did_perform = spotify.perform_auth_grant_flow()

In [168]:
query = spotify.parse_redirected_url('http://localhost:8888/notebooks/BaseApiClass-Copy1.ipynb#access_token=BQBUbtaqgrlUJPpYj-0s-SUaCH1-vDeZ2klSf1CH4lRT4rradMXuDg0ab61z0yunzBCcg1DIPaKWySa0TvwN6cntkMeurWS-2QPSh_72otc1pSqIH49a7gUajA8CTBaJGWdiG2_H0c1eP7Pr77q9&token_type=Bearer&expires_in=3600')
spotify.set_access_token_and_expire(query)
print(spotify.get_access_token())

BQBUbtaqgrlUJPpYj-0s-SUaCH1-vDeZ2klSf1CH4lRT4rradMXuDg0ab61z0yunzBCcg1DIPaKWySa0TvwN6cntkMeurWS-2QPSh_72otc1pSqIH49a7gUajA8CTBaJGWdiG2_H0c1eP7Pr77q9


In [169]:
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {spotify.get_access_token()}"
}
endpoint = "https://api.spotify.com/v1/me/player/currently-playing?market=PL"
#data = urlencode({""})
lookup_url = f"{endpoint}"

r = requests.get(lookup_url, headers=headers)
print(r.json())

{'timestamp': 1616518474553, 'context': {'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1E36R4l1u8NIsU'}, 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1E36R4l1u8NIsU', 'type': 'playlist', 'uri': 'spotify:user:spotify:playlist:37i9dQZF1E36R4l1u8NIsU'}, 'progress_ms': 51442, 'item': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3uL4UpqShC4p2x1dJutoRW'}, 'href': 'https://api.spotify.com/v1/artists/3uL4UpqShC4p2x1dJutoRW', 'id': '3uL4UpqShC4p2x1dJutoRW', 'name': 'Ghostemane', 'type': 'artist', 'uri': 'spotify:artist:3uL4UpqShC4p2x1dJutoRW'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/3gEv1vpsDgaEo4TLuu0Lwz'}, 'href': 'https://api.spotify.com/v1/albums/3gEv1vpsDgaEo4TLuu0Lwz', 'id': '3gEv1vpsDgaEo4TLuu0Lwz', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273d2d74feea8cc2b317638969b', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab6761